In [28]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, LSTM, BatchNormalization
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the path to your dataset
data_path = 'D:/Gerina Mary C/gtzan_dataset/Data/genres_original/'

# Get the list of genres from the directory names
genres = os.listdir(data_path)

# Initialize lists to hold features and labels
data = []
labels = []

# Iterate over each genre
for genre in genres:
    # Get the list of song files for this genre
    song_files = os.listdir(os.path.join(data_path, genre))
    
    # Iterate over each song file
    for song in song_files:
        # Load the song file using librosa
        y, sr = librosa.load(os.path.join(data_path, genre, song))
        
        # Extract features from the audio
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        
        # Append the mean of the features and the genre label to the respective lists
        data.append(np.mean(mfcc, axis=1))
        labels.append(genre)

# Convert features and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the data for the CRNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(len(genres), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train CNN
model.fit(X_train, y_train, epochs=100, batch_size=42, validation_data=(X_test, y_test))

# Extract features from the CNN
X_train_svm = model.predict(X_train)
X_test_svm = model.predict(X_test)
# Define parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=5)
grid_search.fit(X_train_svm, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train SVM with best hyperparameters
best_svm_clf = SVC(**best_params)
best_svm_clf.fit(X_train_svm, y_train)

# Make predictions with the SVM
y_pred = best_svm_clf.predict(X_test_svm)

# Calculate the accuracy of the SVM
accuracy = accuracy_score(y_test, y_pred)

print(f"SVM Accuracy: {accuracy}")

Epoch 1/100
20/20 [==============================] - 2s 21ms/step - loss: 3.3530 - accuracy: 0.1937 - val_loss: 2.0326 - val_accuracy: 0.3100
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 1.9210 - accuracy: 0.3375 - val_loss: 1.9109 - val_accuracy: 0.2950
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 1.7384 - accuracy: 0.3700 - val_loss: 1.8295 - val_accuracy: 0.3450
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 1.6348 - accuracy: 0.4238 - val_loss: 1.7121 - val_accuracy: 0.3650
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 1.5344 - accuracy: 0.4525 - val_loss: 1.7606 - val_accuracy: 0.3500
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 1.4968 - accuracy: 0.4800 - val_loss: 1.6638 - val_accuracy: 0.4150
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 1.4384 - accuracy: 0.4975 - val_loss: 1.6580 - val_accuracy: 0.4000
Epoch 8/100


Epoch 59/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4393 - accuracy: 0.8612 - val_loss: 1.3914 - val_accuracy: 0.5750
Epoch 60/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4504 - accuracy: 0.8475 - val_loss: 1.3763 - val_accuracy: 0.6000
Epoch 61/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4063 - accuracy: 0.8650 - val_loss: 1.5339 - val_accuracy: 0.5650
Epoch 62/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3882 - accuracy: 0.8800 - val_loss: 1.4679 - val_accuracy: 0.5650
Epoch 63/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3913 - accuracy: 0.8650 - val_loss: 1.4516 - val_accuracy: 0.6000
Epoch 64/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3885 - accuracy: 0.8737 - val_loss: 1.4937 - val_accuracy: 0.6000
Epoch 65/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4959 - accuracy: 0.8238 - val_loss: 1.4735 - val_accuracy: 0.5750
Epoch 